In [5]:
import json
import collections
import ast
import pandas as pd
from sklearn.linear_model.logistic import _logistic_regression_path
import random
import numpy as np

In [6]:
json_file = open('yelp_dataset/business.json')

In [7]:
# x = 0
attrs = collections.defaultdict(set)

for line in open('yelp_dataset/business.json','r'):
    curline = json.loads(line)
    if curline['attributes'] != None:
#             if x < 20:
        for param in list(curline['attributes'].keys()):
            paramvalue = attrs[param]
            subparam = ast.literal_eval(curline['attributes'][param])
            if type(subparam) == dict or type(subparam) == type(collections.defaultdict(set)):
                if type(paramvalue) == set:
#                     print('yeet')
                    paramvalue = collections.defaultdict(set)
                if type(paramvalue) == type(collections.defaultdict(set)):
#                     print('yote')
                    for subparamname in list(subparam.keys()):
                        subparamvalue = subparam[subparamname]
#                         print(subparamvalue, type(subparamvalue))
                        if type(subparamvalue) == str:
                            if subparamvalue[:1] == "u":
                                paramvalue[subparamname].add(subparamvalue[1:])
                            else:
                                paramvalue[subparamname].add(subparamvalue)
                        else:
                            paramvalue[subparamname].add(subparamvalue)
                attrs[param] = paramvalue
            else:
#                     print('6eet')
                if not subparam == None:
                    subparamvalue = curline['attributes'][param]
                    if type(subparamvalue) == str:
                        if subparamvalue[:1] == "u":
#                             print(subparamvalue,subparamvalue[1:])
                            paramvalue.add(subparamvalue[1:])
                        else:
                            paramvalue.add(subparamvalue)
                    else:
                         paramvalue.add(subparamvalue)
#                     if subparamvalue[:1] == "u":
#                         print(subparamvalue,subparamvalue[1:],type(subparamvalue))
#                     paramvalue.add(subparamvalue)
                    attrs[param] = paramvalue
#         x += 1



In [8]:
attrs

defaultdict(set,
            {'GoodForKids': {'False', 'True'},
             'RestaurantsReservations': {'False', 'True'},
             'GoodForMeal': defaultdict(set,
                         {'dessert': {False, True},
                          'latenight': {False, True},
                          'lunch': {False, True},
                          'dinner': {False, True},
                          'brunch': {False, True},
                          'breakfast': {False, True}}),
             'BusinessParking': defaultdict(set,
                         {'garage': {False, True},
                          'street': {False, True},
                          'validated': {False, True},
                          'lot': {False, True},
                          'valet': {False, True}}),
             'Caters': {'False', 'True'},
             'NoiseLevel': {"'average'", "'loud'", "'quiet'", "'very_loud'"},
             'RestaurantsTableService': {'False', 'True'},
             'RestaurantsTakeOut':

In [9]:
pd.io.json.json_normalize(attrs, sep='_').to_dict(orient='records')[0]

{'GoodForKids': {'False', 'True'},
 'RestaurantsReservations': {'False', 'True'},
 'Caters': {'False', 'True'},
 'NoiseLevel': {"'average'", "'loud'", "'quiet'", "'very_loud'"},
 'RestaurantsTableService': {'False', 'True'},
 'RestaurantsTakeOut': {'False', 'True'},
 'RestaurantsPriceRange2': {'1', '2', '3', '4'},
 'OutdoorSeating': {'False', 'True'},
 'BikeParking': {'False', 'True'},
 'HasTV': {'False', 'True'},
 'WiFi': {"'free'", "'no'", "'paid'"},
 'Alcohol': {"'beer_and_wine'", "'full_bar'", "'none'"},
 'RestaurantsAttire': {"'casual'", "'dressy'", "'formal'"},
 'RestaurantsGoodForGroups': {'False', 'True'},
 'RestaurantsDelivery': {'False', 'True'},
 'BusinessAcceptsCreditCards': {'False', 'True'},
 'BusinessAcceptsBitcoin': {'False', 'True'},
 'ByAppointmentOnly': {'False', 'True'},
 'AcceptsInsurance': {'False', 'True'},
 'GoodForDancing': {'False', 'True'},
 'CoatCheck': {'False', 'True'},
 'HappyHour': {'False', 'True'},
 'WheelchairAccessible': {'False', 'True'},
 'DogsAllo

In [10]:
def flattenfurther(attrs):
    attrs = pd.io.json.json_normalize(attrs, sep='_').to_dict(orient='records')[0]
    to_delete = set([])
    to_add = set([])

    for attr in attrs:
        if attrs[attr] == {False, True} or attrs[attr] == {False} or attrs[attr] == {True} or attrs[attr] == {'False', 'True'} or attrs[attr] == {'False'} or attrs[attr] == {'True'}:
            continue
        for val in attrs[attr]:
            key = attr + '_' + val.strip('\'')
            to_add.add(key)
        to_delete.add(attr)

    for attr in to_add:
        attrs[attr] = {False, True}

    for attr in to_delete:
        del attrs[attr]

    return attrs

In [11]:
x = flattenfurther(attrs)

In [12]:
x

{'GoodForKids': {'False', 'True'},
 'RestaurantsReservations': {'False', 'True'},
 'Caters': {'False', 'True'},
 'RestaurantsTableService': {'False', 'True'},
 'RestaurantsTakeOut': {'False', 'True'},
 'OutdoorSeating': {'False', 'True'},
 'BikeParking': {'False', 'True'},
 'HasTV': {'False', 'True'},
 'RestaurantsGoodForGroups': {'False', 'True'},
 'RestaurantsDelivery': {'False', 'True'},
 'BusinessAcceptsCreditCards': {'False', 'True'},
 'BusinessAcceptsBitcoin': {'False', 'True'},
 'ByAppointmentOnly': {'False', 'True'},
 'AcceptsInsurance': {'False', 'True'},
 'GoodForDancing': {'False', 'True'},
 'CoatCheck': {'False', 'True'},
 'HappyHour': {'False', 'True'},
 'WheelchairAccessible': {'False', 'True'},
 'DogsAllowed': {'False', 'True'},
 'DriveThru': {'False', 'True'},
 'Corkage': {'False', 'True'},
 'BYOB': {'False', 'True'},
 'Open24Hours': {'False', 'True'},
 'RestaurantsCounterService': {'False', 'True'},
 'GoodForMeal_dessert': {False, True},
 'GoodForMeal_latenight': {Fals

In [13]:
with open('allattrs.txt', 'w') as outfile:
    json.dump(sorted(list(x.keys())),outfile)

In [32]:
pretty_names = {}
for attr in x:
    pretty_string = ""
    for i in range(len(attr)):
        char = attr[i]
        
        if char == '_':
            pretty_string += ' '
            continue
        
        if char.isupper() and not attr[i-1].isupper() and not attr[i+1].isupper() and i != 0:
            pretty_string += ' '
        
        pretty_string += char
    
    pretty_names[attr] = pretty_string
pretty_names

{'GoodForKids': 'Good For Kids',
 'RestaurantsReservations': 'Restaurants Reservations',
 'Caters': 'Caters',
 'RestaurantsTableService': 'Restaurants Table Service',
 'RestaurantsTakeOut': 'Restaurants Take Out',
 'OutdoorSeating': 'Outdoor Seating',
 'BikeParking': 'Bike Parking',
 'HasTV': 'HasTV',
 'RestaurantsGoodForGroups': 'Restaurants Good For Groups',
 'RestaurantsDelivery': 'Restaurants Delivery',
 'BusinessAcceptsCreditCards': 'Business Accepts Credit Cards',
 'BusinessAcceptsBitcoin': 'Business Accepts Bitcoin',
 'ByAppointmentOnly': 'By Appointment Only',
 'AcceptsInsurance': 'Accepts Insurance',
 'GoodForDancing': 'Good For Dancing',
 'CoatCheck': 'Coat Check',
 'HappyHour': 'Happy Hour',
 'WheelchairAccessible': 'Wheelchair Accessible',
 'DogsAllowed': 'Dogs Allowed',
 'DriveThru': 'Drive Thru',
 'Corkage': 'Corkage',
 'BYOB': 'BYOB',
 'Open24Hours': 'Open24 Hours',
 'RestaurantsCounterService': 'Restaurants Counter Service',
 'GoodForMeal_dessert': 'Good For Meal desser

In [33]:
with open('attr_names_map.json', 'w') as outfile:
    json.dump(pretty_names,outfile)